<a href="https://colab.research.google.com/github/AlaseeriRawan/ACMG-PVS1-M-S/blob/main/pct_of_exon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the necessary packages
### complete

In [1]:
# Instillation
!pip install pandas gffutils
!pip install openpyxl
!wget ftp://ftp.ensembl.org/pub/release-112/gtf/homo_sapiens/Homo_sapiens.GRCh38.112.gtf.gz
!gunzip Homo_sapiens.GRCh38.112.gtf.gz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 6.5 MB/s eta 0:00:00
--2024-09-01 06:51:58--  ftp://ftp.ensembl.org/pub/release-112/gtf/homo_sapiens/Homo_sapiens.GRCh38.112.gtf.gz
           => ‘Homo_sapiens.GRCh38.112.gtf.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-112/gtf/homo_sapiens ... done.
==> SIZE Homo_sapiens.GRCh38.112.gtf.gz ... 55529204
==> PASV ... done.    ==> RETR Homo_sapiens.GRCh38.112.gtf.gz ... done.
Length: 55529204 (53M) (unauthoritative)

Homo_sapiens.GRCh38 100%[===================>]  52.96M  12.5MB

In [2]:
import pandas as pd
import requests

In [3]:
# Define the GTF file path
gtf_file = 'Homo_sapiens.GRCh38.112.gtf'
# Define the column names for the GTF file
columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
# Load the GTF file into a pandas DataFrame
df = pd.read_csv(gtf_file, sep='\t', comment='#', names=columns, low_memory=False)

In [4]:
# df.head() data frame of the whole GTF Ensembl data

# Get Ensembl gene and transcript IDs from RefSeq ID, with exon and CDS count
### complete

In [5]:
# @title Transcript Number
# Prompt the user to input the RefSeq ID
refseq_id = input("Please enter the RefSeq ID (e.g., NM_001267550): ")

# Query Ensembl REST API for mapping
server = "https://rest.ensembl.org"
ext = f"/xrefs/symbol/homo_sapiens/{refseq_id}?"

response = requests.get(server + ext, headers={"Content-Type": "application/json"})

if not response.ok:
    response.raise_for_status()

decoded = response.json()

# Initialize variables
gene_id = None
transcript_id = None

# Assign the Ensembl Gene ID and Transcript ID
for item in decoded:
    if 'id' in item and item['type'] == 'gene':
        gene_id = item['id']
        print(f"RefSeq ID: {refseq_id} -> Ensembl Gene ID: {gene_id}")
    elif 'id' in item and item['type'] == 'transcript':
        transcript_id = item['id']
        print(f"RefSeq ID: {refseq_id} -> Ensembl Transcript ID: {transcript_id}")

# Display the assigned values
print(f"\nAssigned Gene ID: {gene_id}")
print(f"Assigned Transcript ID: {transcript_id}")

# Filter the DataFrame for the user-specified gene
gene_df = df[df['attribute'].str.contains(gene_id)]

# Further filter the DataFrame for the specific transcript ID
transcript_df = gene_df[gene_df['attribute'].str.contains(transcript_id)]

# Filter the DataFrame for the specific transcript ID
exons_df = gene_df[(gene_df['feature'] == 'exon') & (gene_df['attribute'].str.contains(transcript_id))]
transcript_ids = gene_df['attribute'].str.extract(r'transcript_id "([^"]+)"')[0].dropna().unique()
cds_df = gene_df[(gene_df['feature'] == 'CDS') & (gene_df['attribute'].str.contains(transcript_id))]


# Count the number of exons
exon_count = len(exons_df)
cds_count = len(cds_df)

# Display the number of exons
print(f"The transcript {transcript_id} has {exon_count} exons and {cds_count} CDS")
print("If the above information is correct, please proceed with the rest of the code below.")

# DATAFRAMES
# gene_df : df for entire gene with all transcrpts
# exons_df : df of just the exons for my particulare transcript
# cds_df :df contains just the CDS for my transcript
# transcript_df : df contains all the data for the transcript including exons, cds, start and end codons and utr regions

Please enter the RefSeq ID (e.g., NM_001267550): NM_006073
RefSeq ID: NM_006073 -> Ensembl Gene ID: ENSG00000186439
RefSeq ID: NM_006073 -> Ensembl Transcript ID: ENST00000334268

Assigned Gene ID: ENSG00000186439
Assigned Transcript ID: ENST00000334268
The transcript ENST00000334268 has 41 exons and 41 CDS
If the above information is correct, please proceed with the rest of the code below.


# Calculate % of the exon

## **positions**:


1.   First CDS (start - end)
2.   Start Codon (start - end)
3.   Other CDS (sum)
4.   Last CDS (start - end)
5.   Stop Codon (start - end)

**total CDS = X**

**total exon = (input CDS number) sum**

**pct of exon = total exon / total CDS * 100**



# Code Execution

In [11]:
# @title Exon Number
# Input the CDS number
cds_number = input("Please enter the exon number you want to analyze (e.g., 15): ")

# Filter for the specified CDS number in the CDS df of the transcript
myCDS_df = cds_df[cds_df['attribute'].str.contains(f'exon_number "{cds_number}"')]

# Extract the start, end, and attribute information for the selected cds
cds_start = myCDS_df['start'].values[0]
cds_end = myCDS_df['end'].values[0]
cds_length = cds_end - cds_start + 1  # Calculate exon length
cds_attributes = myCDS_df['attribute'].values[0]

# Extract the start, end, and attribute of transcript
transcript_start = transcript_df['start'].values[0]
transcript_end = transcript_df['end'].values[0]
transcript_length = transcript_end - transcript_start + 1

# Calculate the total length of all exons
cds_lengths = cds_df['end'] - cds_df['start'] +1
full_cds_length = cds_lengths.sum() + 3

# Calculate the percentage of the selected exon relative to the entire gene length
exon_percentage = (cds_length / transcript_length) * 100

# Display the start, end, length, and full attributes
print(f"Exon {cds_number} Start: {cds_start}")
print(f"Exon {cds_number} End: {cds_end}")
print(f"Exon {cds_number} Length: {cds_length}")

# Display the start, end, and length of the transcript
print(f"Transcript Start: {transcript_start}")
print(f"Transcript End: {transcript_end}")
print(f"CDS Length: {full_cds_length}")

# pct manual
pct_ex = (cds_length / full_cds_length) * 100
print(f"Exon {cds_number} is {pct_ex}% of the total gene length.")

if pct_ex > 9.99:
  print("PVS1_STRONG")
else:
    print("PVS1_MODERATE")

Please enter the exon number you want to analyze (e.g., 15): 25
Exon 25 Start: 123279056
Exon 25 End: 123279082
Exon 25 Length: 27
Transcript Start: 123216339
Transcript End: 123636950
CDS Length: 2190
Exon 25 is 1.2328767123287672% of the total gene length.
PVS1_MODERATE
